# 00_TEMPLATE_test_any_agent\nUniversal notebook to test any agent with a consistent contract.

In [ ]:
!pip -q install -r requirements.txt

In [ ]:
AGENT_IMPORT = "agents.website_locator.agent"  # change per agent\nAGENT_CLASS  = "WebsiteLocatorAgent"           # change per agent\n\nRUN_CONFIG = {\n    "mode": "TEST",\n    "network": "OFF",\n    "timezone": "America/New_York",\n    "artifact_dir": "/content/artifacts",\n    "run_id": None,\n}\n\nFIXTURE_ROOT = "fixtures"

In [ ]:
from common.fixtures import FixtureStore\nfixtures = FixtureStore(FIXTURE_ROOT)\n\nINPUT_PAYLOAD = {\n    "courthouse_name": "Example County Clerk",\n    "courthouse_address": "123 Main St, Exampletown, PA 19000",\n    "state": "PA",\n    "county": "Example",\n    "seed_url": "https://example.gov",\n    "fixture_keys": {\n        "homepage_html": "html_snapshots/example_clerk_home.html"\n    }\n}

In [ ]:
import importlib\n\nmod = importlib.import_module(AGENT_IMPORT)\nAgentCls = getattr(mod, AGENT_CLASS)\n\n# Instantiate with fixtures if supported\ninit_vars = AgentCls.__init__.__code__.co_varnames\nagent = AgentCls(fixtures=fixtures) if "fixtures" in init_vars else AgentCls()\n\nresult = agent.run(INPUT_PAYLOAD, RUN_CONFIG)\nresult["status"], result["run_id"]

In [ ]:
import json\nprint(json.dumps(result, indent=2)[:4000])

In [ ]:
from pathlib import Path\nartifact_dir = Path(RUN_CONFIG["artifact_dir"]) / result["agent_name"] / result["run_id"]\nsorted([str(p) for p in artifact_dir.rglob("*") if p.is_file()])[:50]

In [ ]:
assert result["status"] in ("ok","blocked","error")\nassert "agent_name" in result and "run_id" in result\nassert isinstance(result["artifacts"], list)\nassert isinstance(result["errors"], list)\nif result["status"] == "blocked":\n    assert any(e.get("code") == "POLICY_BLOCK" for e in result["errors"])